<a href="https://colab.research.google.com/github/kimsooyoung/practical_jetson_examples/blob/main/temp/%5BPractical%20Jetson%20%231-3%5D%20Digits_Recognition_with_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Practical Exercise with MNIST Example #1-3 - "Train with CNN Model"**

**[!important] Check your Runtime type bf running this example (Runtime > Change runtime type)**

In [ ]:
import torch
import torch.nn.init
import torchvision.datasets as datasets
import torchvision.transforms as transforms

# **GPU usability check**

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

## **Define Hyperparameters**

In [ ]:
learning_rate = 0.001
training_epochs = 15
batch_size = 100

## **Train/Test data through Dataloader**

In [ ]:
mnist_train = datasets.MNIST(root='MNIST_data/',         # root: download path
                        train=True,                      # train dataset
                        transform=transforms.ToTensor(), # 텐서로 변환
                        download=True)                   # download = True: If no data on the root, download it.

mnist_test = datasets.MNIST(root='MNIST_data/',
                        train=False,                     # test dataset
                        transform=transforms.ToTensor(),
                        download=True)

# Print dataset size
print(len(mnist_train))
print(len(mnist_test))

60000
10000


## **Define DataLoader**

* `DataLoader` reads dataset and fetches data in batch. This makes it more easier and efficient to train the model.

In [ ]:
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, # Push train dataset into DataLoader
                                        batch_size=batch_size, # batch size
                                        shuffle=True,          # random suffle option
                                        drop_last=True)        # whether to discard the last batch if it is less than batch_size. / If True, discard the last remaining data.

## **3 Layer CNN Model**

In [ ]:
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()


        #### Layer1 ####
        # Image Input shape = (Data Length, Width, Height, Channel)
        # Input shape = (N, 28, 28, 1)
        # Conv = (N, 28, 28, 32)
        # Pool = (N, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        #### Layer2 ####
        # Input shape = (N, 14, 14, 32)
        # Conv = (N, 14, 14, 64)
        # Pool = (N, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))

        #### Layer3 ####
        # Fully connected layer
        # Input shape = (N, 7, 7, 64)
        # Output = (N, 10)
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)

        # Initialize weights with Xavier initialization to the fully connected layer only
        torch.nn.init.xavier_uniform_(self.fc.weight)

    # forward pass
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # flatten for fully connected layer
        out = self.fc(out)
        return out

# Model instance with GPU
model = CNN().to(device)

## **Optimizer and Loss Function**

In [ ]:
# Define CrossEntropyLoss suitable for multi-class classification
criterion = torch.nn.CrossEntropyLoss().to(device)

# Using Adam Optimization Algorithm
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## **Main Train Loop**

In [ ]:
total_batch = len(data_loader)

for epoch in range(training_epochs):

    avg_cost = 0

    for X, Y in data_loader:
        X = X.to(device) # X : input image data
        Y = Y.to(device) # Y : Answer label

        optimizer.zero_grad()

        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()

        optimizer.step()
        avg_cost += cost / total_batch

    # Outputs the average loss of Epoch.
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.221689969
[Epoch:    2] cost = 0.0599666983
[Epoch:    3] cost = 0.0435661599
[Epoch:    4] cost = 0.0360821448
[Epoch:    5] cost = 0.0293143354
[Epoch:    6] cost = 0.0251823589
[Epoch:    7] cost = 0.02032586
[Epoch:    8] cost = 0.0171206556
[Epoch:    9] cost = 0.0148831559
[Epoch:   10] cost = 0.0131219123
[Epoch:   11] cost = 0.0110590309
[Epoch:   12] cost = 0.00921344664
[Epoch:   13] cost = 0.00751071097
[Epoch:   14] cost = 0.00662531937
[Epoch:   15] cost = 0.00646132929


## **Model Accuracy Validation**

In [ ]:
# Disable gradient calculation bcs will not proceed with training to update the weights of the Model
with torch.no_grad():

    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)  # .float(): converts pixel values to floating points
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)

    # Compare the predicted with the correct answer label for each test data
    correct_prediction = (torch.argmax(prediction, 1) == Y_test)

    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9824000000953674


/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


## **Save Trained Model**

In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'epoch': epoch + 1,
    'avg_cost': avg_cost.item(),
    'learning_rate': learning_rate,  # Save Hyperparameter
    'batch_size': batch_size,        # Save Hyperparameter
}, './CNN_trained_model.pth')